In [39]:
import os
import numpy as np

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend
from tensorflow import distribute
import pickle
import data_processing as dp

In [40]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten, UpSampling2D, Conv2D

In [41]:
BATCH_SIZE = 32 
learning_rate = 0.005
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
image_size = (32, 32, 3)
sampling = 3 

In [42]:
(x_train, y_train), (x_test, y_test) = dp.load_data_keras("../Data")

In [43]:
#(training_images, training_labels) , (validation_images, validation_labels) = tf.keras.datasets.cifar10.load_data()
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  input_images = UpSampling2D(size=(sampling, sampling))(input_images)
  output_ims = preprocess_input(input_images)
  return output_ims
strategy = distribute.get_strategy()
with strategy.scope():
  vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0] * sampling, image_size[1] * sampling, image_size[2]))
  x = vgg_model.output
  x = Conv2D(1024, (3, 3), padding='same', activation='relu')(x)
  x = Conv2D(2048, (3, 3), padding='same', activation='relu')(x)
model = Model(inputs=vgg_model.input, outputs=x)
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 96, 96, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 96, 96, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 48, 48, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 48, 48, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 24, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 24, 24, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 24, 24, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 24, 24, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 12, 12, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 12, 12, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 12, 12, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 3, 3, 1024)     │     4,719,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 3, 3, 2048)     │    18,876,416 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,310,720 (146.14 MB)

 Trainable params: 38,310,720 (146.14 MB)

 Non-trainable params: 0 (0.00 B)

In [44]:
features_train = model.predict(preprocess_image_input(x_train))

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 618s 395ms/step


In [45]:
features_test = model.predict(preprocess_image_input(x_test))

313/313 ━━━━━━━━━━━━━━━━━━━━ 124s 395ms/step


In [46]:
features_train.shape

(50000, 3, 3, 2048)

In [47]:
n_features = features_train.shape[1:]
log_model = Sequential([
    Input(shape=n_features),
    Flatten(),
    Dense(10, activation='softmax')
])
log_model.compile(optimizer=Adam(learning_rate=learning_rate), loss=SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
log_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)             │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │       184,330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 184,330 (720.04 KB)

 Trainable params: 184,330 (720.04 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
checkpoint = ModelCheckpoint(
    'model/vgg_model.keras',             # Path where to save the model
    monitor='val_accuracy',       # Metric to monitor
    save_best_only=True,          # Save only the best model
    mode='max',                   # Mode for choosing the best value ('max' since higher accuracy is better)
    verbose=1                     # Verbose output to see when checkpoints are saved
)
log_model.fit(features_train, y_train, batch_size=BATCH_SIZE, epochs=10, validation_data=(features_test, y_test), callbacks=[checkpoint])

Epoch 1/10
1556/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6112 - loss: 17.0589
Epoch 1: val_accuracy improved from -inf to 0.72690, saving model to model/vgg_model.keras
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6116 - loss: 17.0375 - val_accuracy: 0.7269 - val_loss: 13.3906
Epoch 2/10
1530/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7726 - loss: 9.4317
Epoch 2: val_accuracy did not improve from 0.72690
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7726 - loss: 9.4502 - val_accuracy: 0.7241 - val_loss: 17.1541
Epoch 3/10
1537/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8119 - loss: 8.6025
Epoch 3: val_accuracy improved from 0.72690 to 0.74340, saving model to model/vgg_model.keras
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8118 - loss: 8.6114 - val_accuracy: 0.7434 - val_loss: 16.2155
Epoch 4/10
1553/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8433 - loss: 7.1138
Epoch 4: val_accuracy did not improve from 0.74340

In [49]:
from sklearn.metrics import classification_report
probabilities = log_model.predict(features_test, batch_size=128)

probabilities = np.argmax(probabilities, axis = 1)

print(classification_report(y_test, probabilities, target_names= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'], digits=4))

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step
              precision    recall  f1-score   support

    airplane     0.8504    0.8070    0.8281      1000
  automobile     0.9140    0.8710    0.8920      1000
        bird     0.8106    0.7020    0.7524      1000
         cat     0.5447    0.6400    0.5885      1000
        deer     0.8409    0.5920    0.6948      1000
         dog     0.5609    0.7970    0.6584      1000
        frog     0.7865    0.8030    0.7947      1000
       horse     0.8339    0.7730    0.8023      1000
        ship     0.8938    0.8840    0.8889      1000
       truck     0.8925    0.8880    0.8902      1000

    accuracy                         0.7757     10000
   macro avg     0.7928    0.7757    0.7790     10000
weighted avg     0.7928    0.7757    0.7790     10000

